In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import os
import json
import re
from datasets import Dataset
from torch.utils.data import DataLoader

from transformers import T5ForConditionalGeneration, T5Tokenizer, MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer

In [ ]:
drive.mount('/content/drive')
!cd /content/drive/MyDrive/Shared-Tasks/Num-Eval

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Shared-Tasks/Num-Eval

/content/drive/MyDrive/Shared-Tasks/Num-Eval


In [ ]:
def file_loader(json_path):
  with open(json_path) as f:
    contents = f.read()
  return json.loads(contents)

In [ ]:
# numerical_reasoning_dev = "Dev_Numerical_Reasoning.json"
# numerical_reasoning_train = "Train_Numerical_Reasoning.json"

# numerical_data_dev_path = os.path.join("Dataset", numerical_reasoning_dev)
# numerical_data_train_path = os.path.join("Dataset", numerical_reasoning_train)

# numerical_data_train = file_loader(numerical_data_train_path)
# df_train = pd.DataFrame.from_dict(numerical_data_train)

# numerical_data_dev = file_loader(numerical_data_dev_path)
# df_dev = pd.DataFrame.from_dict(numerical_data_dev)

In [ ]:
# df_dev['context'] = df_dev['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x, 1).strip())
# df_train['context'] = df_train['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x, 1).strip())

In [ ]:
# def process_input(item):

#   headline = item["masked headline"]
#   return f"{item['context']}\n\n Fill in the blank: {headline}"

In [ ]:
# df_dev['t5-input'] = df_dev.apply(lambda x: process_input(x),axis=1)

In [ ]:
# df_train['t5-input'] = df_train.apply(lambda x: process_input(x),axis=1)

In [ ]:
# df_train['ans'] = df_train['ans'].astype(str)
# df_dev['ans'] = df_dev['ans'].astype(str)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_id="google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
 # train_data = Dataset.from_pandas(df_train)
# dev_data = Dataset.from_pandas(df_dev)

In [ ]:
# test_dev = dev_data.train_test_split(test_size=0.5, seed=41)

In [ ]:
# test_data = test_dev['test']
# dev_data = test_dev['train']

In [ ]:
# dev_data.save_to_disk('./Dataset/dev')
# test_data.save_to_disk('./Dataset/test')
# train_data.save_to_disk('./Dataset/train')

In [ ]:
from datasets import load_from_disk
# test_data = load_from_disk('./Dataset/test')
train_data = load_from_disk('./Dataset/train')
dev_data = load_from_disk('./Dataset/dev')

In [ ]:
def collator(batch):

  input = batch['t5-input'] #load original sentences
  label = batch['ans'] #load noisy sentences
  inputs = tokenizer(input, text_target=label, return_tensors="pt", max_length = 512, padding='max_length',truncation=True) #tokenized sentences

  return inputs

In [ ]:
train_tokenized = train_data.map(collator, remove_columns=train_data.column_names, batch_size=8, num_proc=4, batched=True)
dev_tokenized = dev_data.map(collator, remove_columns=dev_data.column_names, batch_size=8, num_proc=4, batched=True)


Map (num_proc=4):   0%|          | 0/21157 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1286 [00:00<?, ? examples/s]

In [ ]:
#define evaluation steps based on total dataset size
TRAIN_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 5e-5
data_size = len(train_tokenized)
EVAL_STEPS=int(data_size/(TRAIN_BATCH_SIZE*2))
SAVE_STEPS= EVAL_STEPS




In [ ]:
#define training argument object
training_args = Seq2SeqTrainingArguments(
  output_dir="./Outputs",
  group_by_length=True,
  per_device_train_batch_size=TRAIN_BATCH_SIZE,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=EPOCHS,
  fp16=False,
  save_steps=SAVE_STEPS,
  eval_steps=EVAL_STEPS,
  logging_steps=EVAL_STEPS,
  learning_rate=LEARNING_RATE,
  warmup_steps=100,
  save_total_limit=2,
)


#define training data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, return_tensors='pt')


#define Seq2Seq Training object
trainer = Seq2SeqTrainer(
    model=model.cuda(),
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
    tokenizer=tokenizer
)

In [ ]:
outputDir = "./Outputs"
saved_model = f"{outputDir}/checkpoint-5289"
trainer.train(saved_model)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
10578,0.001500,0.001323
15867,0.001100,0.001166


In [ ]:

trainer.save_model(f"{outputDir}/Trial-base")